In [1]:

import numpy as np
import os
import random
import copy
import pandas
import time
import pickle
from psychopy import monitors, visual, core, event


In [ ]:
rng = np.random.default_rng(seed=None)
linejitter_arr =(np.ones((30,30,2))- rng.random((30,30,2))*2)/80
# file_name = os.getcwd() + '/stim_jitter.npy'
# np.save(file_name,linejitter_arr)



In [3]:
def generateStimCoordinates(gridcol, gridrow, jitter):
    """
    Generates coordinates used to draw the stimulus lines
    """
    # Calculate spacing of grid
    x_spacing = 1.0 / (gridcol)
    y_spacing = 1.0 / (gridrow)

    # Create an array to store coordinates: every point (x,y) of the grid for every quadrant (4) 
    coord_array = np.empty(shape = (gridcol,gridrow,2,4),dtype= 'object')
    quadrant_set = [[-1,1],[1,1],[1,-1],[-1,-1]]

    # Generate grid coord per quadrant
    for i,quad in enumerate(quadrant_set):
        # Per row
        for row in range(gridrow):
            # Per column
            for col in range (gridcol):

                grid_x = col * x_spacing  # grid points
                grid_y = row * y_spacing  

                grid_x = grid_x * quad[0]   # quadrant position
                grid_y = grid_y * quad[1]   # quadrant position

                coord_array[col,row,0,i] = grid_x  # Store them in big ass array
                coord_array[col,row,1,i] = grid_y

    # flip the matrices so that they have the 'orientation' corresponding to the quadrant 
    coord_array[:,:,0,0] = np.flip(coord_array[:,:,0,0], axis=0) 
    coord_array[:,:,1,0] = np.flip(coord_array[:,:,1,0], axis=0) 

    coord_array[:,:,0,2] = np.flip(coord_array[:,:,0,2], axis=1) 
    coord_array[:,:,1,2] = np.flip(coord_array[:,:,1,2], axis=1) 

    coord_array[:,:,0,3] = np.flip(coord_array[:,:,0,3], axis=(0,1)) 
    coord_array[:,:,1,3] = np.flip(coord_array[:,:,1,3], axis=(0,1)) 

    #  Add jitter
    for i in range(4):
        coord_array[:,:,:,i] =  coord_array[:,:,:,i] + jitter[:gridcol,:gridrow,:]
  
    return coord_array



def generateStim(linelength, linewidth ,coord_array, colour, catch_colour, size, fixdistance, localizer = False ):
    """ 
    Generate an ElementArrayStim object consisting of lines on the coordinates for stimulus for each quadrant
    Then four more arrays are created each representing a catch trial in a separate quadrant
    Returns a 5x4 array of lists with each list containing the line stimuli for a quadrant
    If localizer is set to True then it generates a grid in the upper or lower visual field instead of quadrants
    """
    # This 
    if localizer:
        sections = 2
        dist = fixdistance
        quads = [[0,1],[0,-1]]
    else:
        sections = 4
        # This is to calculate the distance to the fixation cross
        dist = np.sqrt(np.square(fixdistance)/2) #pythagoras
        quads = [[-1,1],[1,1],[1,-1],[-1,-1]]

    # Get size of grid
    gridcol = np.shape(coord_array)[0]
    gridrow = np.shape(coord_array)[1]

    # Init arrays to store line objects per quadrant and also for every possible catch trial 
    line_stimuli = np.empty((2,sections), dtype=object)   # 2 types (0 is normal white lines, 1 is catch stim ) and 4 quadrants
 
    for quad in range(sections):
        n_lines = gridcol*gridrow
        xys = np.empty((n_lines,2)) #coordinates to pass to elementArrayStim
        it_count = 0 # idk man... this is a work around because im too lazy to completely change the structure that I used before i.e. two nested loops per column (below)
        for row in range(gridrow):
            for col in range(gridcol):

                # Define stim coordinates
                the_x = (coord_array[col,row,0,quad] * size) + (quads[quad][0]*dist)  # size and distance from fixation are all in here 
                the_y = (coord_array[col,row,1,quad] * size) + (quads[quad][1]*dist)              
                                                                              
                xys[it_count,0] = the_x
                xys[it_count,1] = the_y
                it_count += 1
        sizes = np.atleast_2d([linelength,linewidth]).repeat(repeats=n_lines, axis=0)
        # Normal stimuli
        line_stimuli[0,quad] = visual.ElementArrayStim(win, units='pix',elementTex=None, elementMask='sqr', xys= xys,
                                           nElements=n_lines, sizes=sizes, colors=(1.0, 1.0, 1.0), colorSpace='rgb')
        # Catch stimuli
        line_stimuli[1,quad] = visual.ElementArrayStim(win, units='pix',elementTex=None, elementMask='sqr', xys= xys,
                                           nElements=n_lines, sizes=sizes, colors=catch_colour, colorSpace='rgb')

    return line_stimuli


def drawStim(line_stimuli, quad, catch, task):
    """ 
    Draws the stimuli in the correct quadrant
    Catch = 0 means normal color catch = 1 means catch
    """
    # Evaluate the task and make the apropriate catch
    if task == 'stim':
        line_stim = line_stimuli[catch,quad] 
    elif task == 'cross':
        line_stim = line_stimuli[0,quad]

    # Draw 
    line_stim.draw()


In [4]:
system_monitor = monitors.Monitor('cap_lab_monitor')   
# #initialize window#


In [ ]:
win = visual.Window(fullscr=True,color= (-1, -1, -1), colorSpace = 'rgb', units = 'pix', monitor= system_monitor)
win.mouseVisible = False
grid = generateStimCoordinates(gridcol=12, gridrow=10, jitter=linejitter_arr)
stimset = generateStim(linelength=35,linewidth=2, coord_array=grid, colour='white',catch_colour= 'red',
                        size=276, fixdistance=134)

keys = event.getKeys()
    # Show the form
while not 'return' in keys:
    drawStim(stimset,0,0,'stim')
    win.flip()
    keys = event.getKeys()
    if 'escape' in keys:
        break
core.quit()